In [2]:
# NLP Lecture @ Strive School - 21st July 2021
# CHATBOT with Pytorch

'''
Chatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.

Imagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.

Create a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.

The final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.

Example
A: Hello MedAssistant.
B: Hello. How can I help you?
A: I don't feel well.
B: Do you have any symptoms?
A: I have cough and nausea.
B: Do you want to book an appointment?
A: Yes, please, for tomorrow.


Info:
- Feel free to change or arrange a new dataset of intents
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)
'''

In [2]:
import numpy as np
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

# STEP 0: find intents patterns

with open('intents_health.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
patterns = []

for intent in intents['intents']:
# TBD: loop through each sentence in our intents patterns, create a list of tags and define the patterns
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        patterns.append((w, tag))



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rujek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# STEP 1: Pre-process of the input

# lower case? stemming? stopwords?
# TBD
ignore_words = ['?', '!', '.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [4]:
# STEP 2: Define training data through a bag of words

X_train = []
y_train = []
for (pattern_sentence, tag) in patterns:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [6]:
# STEP 3: Configure the neural network

# define each parameter that is equal to 0 using an empirical value or a value based on your experience
# TBD

num_epochs = 100
batch_size = 16
learning_rate = 0.002
input_size = len(X_train[0])
hidden_size = 16
output_size = len(tags)

# STEP 4: Train the model

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Define loss and optimizer: which one is the best one?
# TBD
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

In [7]:
# STEP 5: Save the model

# TBD: name and save the model
file = "trained.pth"
torch.save(data, file)

In [10]:
# STEP 6: Test the model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# TBD: Load the intents file
with open('intents_health.json', 'r') as json_data:
    intents = json.load(json_data)

# TBD: Retrieve the model and all the sizings
data = torch.load(file)

# TBD: build the NN
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

# TBD: prepare a command-line conversation (don't forget something to make the user exit the script!)

bot_name = "Health bot"
print("Let's have a chat! (type 'exit' to stop chatting!)")

while True:
    sentence = input("You: ")
    if sentence == "exit":
        print("Thanks for joining me!")
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    print(prob)
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent['tag']:
                #print(f"{bot_name}: {random.choice(intent['responses'])}")
                print(f"{bot_name}: {(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand. Try to be more specific")

Let's have a chat! (type 'exit' to stop chatting!)
tensor(0.5514, grad_fn=<SelectBackward0>)
Health bot: I do not understand. Try to be more specific
tensor(0.9485, grad_fn=<SelectBackward0>)
Health bot: ['Hey!', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?', 'Hi there, how are you feeling?']
tensor(0.5514, grad_fn=<SelectBackward0>)
Health bot: I do not understand. Try to be more specific
tensor(0.7766, grad_fn=<SelectBackward0>)
Health bot: ['Which symptoms do you have?', 'Do you have severe symptoms?', 'For nearest hospital call or text xxxx1234xxx', 'Yes, we have appointment today']
tensor(0.5514, grad_fn=<SelectBackward0>)
Health bot: I do not understand. Try to be more specific
Thanks for joining me!
